In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import gymnasium as gym
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
import numpy as np
import matplotlib.pyplot as plt
import os
from plot_utils import *
from utils import *
from agent_env_utils import *
from config import color_maps, background_quadrants, background_specs
import warnings
from scene_helpers import get_position_range, create_config, create_directories, finalize_quadrant



In [5]:
# Define directories
data_dir = "../benchmark/quantity/discrete_counting"
final_dir = create_directories(data_dir, empty=True)

# Initialize pair of colors
color_keys = list(color_maps.keys())
random.seed(42)
random.shuffle(color_keys)

# Define ranges and object parameters
object_shapes = ["sphere", "cube"]
object_size_ranges = {
    "sphere": (0.02, 0.04),
    "cube": (0.02, 0.04),
}
num_objects_range = (1,7)
setting_num = 100

count = 0

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    outputs = []
    for background in trange(4):
        for color_idx in range(len(color_keys)):  # Adjust the range as needed
            color = color_keys[color_idx]
            for num_objects in range(num_objects_range[0], num_objects_range[1] + 1):  
                # Generate random configurations for objects
                for r in range(setting_num):
                    count += 1
                    configs = []
                    object_types = []
                    object_sizes = []
                    for _ in range(num_objects):
                        obj_type = random.choice(object_shapes)
                        object_types.append(obj_type)
                        
                        position_range = background_specs[background] #get_position_range(background_quadrants, background, scale=1)
                        
                        
                        size_range = object_size_ranges[obj_type]
                        size = np.random.uniform(size_range[0], size_range[1])
                        object_sizes.append(size)
                        configs.append(create_config(obj_type, color_maps[color], size, position_range))
                        

                    configs_collate = collate_infos(configs)

                    env.register_configures(configs_collate)
                    
                    _ = env.reset(options={"reconfigure": True})

                    env.initialize_objects(background=background)
                    
                    obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
                    
                    save_dir_final = f"{final_dir}/scene_{count}.png"
                    views = [f"front{background}", f"side{background}", f"top{background}"]
                    collect_and_save(env, save_dir_final, steps=1, mode=views)
                    poses_final = env.get_important_obj_poses()
                    sources = [f"{save_dir_final}_{view}" for view in views]
                    
                    outputs.append({
                        "source": sources,
                        "color": color,
                        "num_objects": num_objects,
                        "background": background,
                        "round": r,
                        "sizes": [np.round(size, 4).tolist() for size in object_sizes],
                        "obj_names": object_types,
                        
                        "poses_final": [np.round(item, 4).tolist() for item in poses_final],
                    })

    write_out(f"{data_dir}/index.jsonl", outputs)

del env

count

[2024-10-03 01:48:44.041] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.
100%|██████████| 1/1 [00:31<00:00, 31.10s/it]


35

In [6]:
np.random.uniform(0.02, 0.05)

0.021893383412180303